In [28]:
import pandas as pd

import pickle



In [29]:
import pandas as pd
df = pd.read_pickle('data/csi300.pkl')

In [30]:
df

,dt,code,close,high,low,open,to,vol,label
0,2022-11-01,000001.SZ,0.018500,0.018292,0.018760,0.018052,0.162606,0.223802,-0.003756
1,2022-11-01,000002.SZ,0.024635,0.024345,0.024638,0.024083,0.181035,0.193991,0.003601
2,2022-11-01,000063.SZ,0.041155,0.040791,0.042311,0.040859,0.070436,0.045882,0.015908
3,2022-11-01,000069.SZ,0.006308,0.006262,0.006240,0.006088,0.021371,0.073221,0.020572
4,2022-11-02,000001.SZ,0.018423,0.018292,0.019002,0.018497,0.153478,0.211124,-0.018036
...,...,...,...,...,...,...,...,...,...
171,2022-12-29,000069.SZ,0.008135,0.008394,0.008495,0.008504,0.043796,0.119776,0.009425
172,2022-12-30,000001.SZ,0.023289,0.023241,0.023994,0.023193,0.119720,0.132547,0.011335
173,2022-12-30,000002.SZ,0.032981,0.032835,0.033998,0.033225,0.104168,0.083424,0.076047
174,2022-12-30,000063.SZ,0.047712,0.047453,0.049337,0.047508,0.064096,0.036075,0.023027


In [31]:
print('ssss')

ssss


In [32]:
df_ = pd.read_pickle('data/valid_csi300.pkl')

In [33]:
df_.reset_index(inplace=True)
df_.rename(columns={'datetime': 'dt'}, inplace=True)
df_.rename(columns = {'instrument':'code'}, inplace=True)

In [34]:
df_ = df_[['dt', 'code', 'KMID','KLEN',	'KMID2','KUP','KUP2','LABEL0']]

In [35]:
df_

,dt,code,KMID,KLEN,KMID2,KUP,KUP2,LABEL0
0,2017-12-04,SH600000,0.233334,-1.284120,0.758802,-0.747315,-0.216976,-1.067304
1,2017-12-04,SH600008,-1.456682,0.149920,-1.233051,-0.404646,-0.542506,0.209689
2,2017-12-04,SH600009,-0.156256,-0.636687,-0.222577,0.124836,0.651043,-2.116940
3,2017-12-04,SH600010,0.456733,-0.307101,0.505868,0.204933,0.361681,-0.131998
4,2017-12-04,SH600015,0.129305,-1.089662,0.303532,-0.144860,1.084999,-0.750083
...,...,...,...,...,...,...,...,...
76438,2018-12-28,SZ300251,-2.353814,2.398624,-0.922465,1.793636,0.106381,1.418828
76439,2018-12-28,SZ300296,0.551733,-0.254601,0.590176,0.453886,0.602806,-2.755846
76440,2018-12-28,SZ300308,-1.384341,1.314281,-0.732111,-0.236152,-0.648825,-2.075338
76441,2018-12-28,SZ300408,-0.317776,-0.605141,-0.440589,-0.806837,-0.804999,-3.124002


In [36]:
import numpy as np
import multiprocessing as mp
from tqdm import tqdm

feature_cols = ['KMID','KLEN','KMID2','KUP','KUP2']

def cal_pccs(x, y, n):
    sum_xy = np.sum(np.sum(x*y)) #요소간 곱한 것을 모두 더하기 위함임. e.g. [1,2,3]*[1,2,3] -> 근데 굳이 이렇게 해야하나...
    sum_x = np.sum(np.sum(x))
    sum_y = np.sum(np.sum(y))
    sum_x2 = np.sum(np.sum(x*x))
    sum_y2 = np.sum(np.sum(y*y))
    pcc = (n*sum_xy-sum_x*sum_y)/np.sqrt((n*sum_x2-sum_x*sum_x)*(n*sum_y2-sum_y*sum_y))
    return pcc

def calculate_pccs(xs, yss, n): # ref_dict[code], ref_dict, n
    '''
    각 stock code의 각 feature 별로 상관관계가 계산된다. 이 때 tmp_res에는 각 주식 code별 상관관계가 저장되며 임시 tmp_res는 results에 저장된다.
     '''
    result = []
    for name in yss:
        ys = yss[name]
        tmp_res = []
        for pos, x in enumerate(xs): # pos는 feature의 index, x는 feature의 20일 동안의 데이터
            y = ys[pos] # 하나의 feature에 대해서 20일 어치의 데이터를 가져온다.
            tmp_res.append(cal_pccs(x, y, n)) # x: 주식 code, y: features, n: 참조할 과거 시계열 길이 -> pearson 상관관계 값을 구한다.
        result.append(tmp_res) # 각code 별로 진행.
    return np.mean(result, axis=1) # mean of all pccs -> 6개의 feature에 대한 평균 pccs -> 이렇게 하면 안되지 않니... 어떻게 했지 

def stock_cor_matrix(ref_dict, codes, n, processes=1):
    if processes > 1:
        pool = mp.Pool(processes=processes)
        args_all = [(ref_dict[code], ref_dict, n) for code in codes] # 전체 코드에 대해서 각각의 코드에 대한 pccs를 구하고 튜플로서 저장
        results = [pool.apply_async(calculate_pccs, args=args) for args in args_all] # 저장된 튜플(x,y,n)이 있으면 그것에 대해서 멀
        output = [o.get() for o in results]
        data = np.stack(output) #  하나의 배열로서 합친다.
        return pd.DataFrame(data=data, index=codes, columns=codes)
    data = np.zeros([len(codes), len(codes)])
    for i in tqdm(range(len(codes))): # 행은 code, 열은 feature
        data[i, :] = calculate_pccs(ref_dict[codes[i]], ref_dict, n)
    return pd.DataFrame(data=data, index=codes, columns=codes)

path1 = "./data/valid_csi300.pkl" 


In [37]:
df_

,dt,code,KMID,KLEN,KMID2,KUP,KUP2,LABEL0
0,2017-12-04,SH600000,0.233334,-1.284120,0.758802,-0.747315,-0.216976,-1.067304
1,2017-12-04,SH600008,-1.456682,0.149920,-1.233051,-0.404646,-0.542506,0.209689
2,2017-12-04,SH600009,-0.156256,-0.636687,-0.222577,0.124836,0.651043,-2.116940
3,2017-12-04,SH600010,0.456733,-0.307101,0.505868,0.204933,0.361681,-0.131998
4,2017-12-04,SH600015,0.129305,-1.089662,0.303532,-0.144860,1.084999,-0.750083
...,...,...,...,...,...,...,...,...
76438,2018-12-28,SZ300251,-2.353814,2.398624,-0.922465,1.793636,0.106381,1.418828
76439,2018-12-28,SZ300296,0.551733,-0.254601,0.590176,0.453886,0.602806,-2.755846
76440,2018-12-28,SZ300308,-1.384341,1.314281,-0.732111,-0.236152,-0.648825,-2.075338
76441,2018-12-28,SZ300408,-0.317776,-0.605141,-0.440589,-0.806837,-0.804999,-3.124002


In [38]:
#import pickle
#df1 = pickle.load(open(path1, 'rb'), encoding='utf-8')
#prev_date_num Indicates the number of days in which stock correlation is calculated
prev_date_num = 20
date_unique=df_['dt'].unique()
stock_trade_data=date_unique.tolist()
stock_trade_data.sort()
stock_num=df_.code.unique().shape[0]


In [39]:
stock_num

368

In [40]:
df_['dt'].unique()[20:]

<DatetimeArray>
['2018-01-02 00:00:00', '2018-01-03 00:00:00', '2018-01-04 00:00:00',
 '2018-01-05 00:00:00', '2018-01-08 00:00:00', '2018-01-09 00:00:00',
 '2018-01-10 00:00:00', '2018-01-11 00:00:00', '2018-01-12 00:00:00',
 '2018-01-15 00:00:00',
 ...
 '2018-12-17 00:00:00', '2018-12-18 00:00:00', '2018-12-19 00:00:00',
 '2018-12-20 00:00:00', '2018-12-21 00:00:00', '2018-12-24 00:00:00',
 '2018-12-25 00:00:00', '2018-12-26 00:00:00', '2018-12-27 00:00:00',
 '2018-12-28 00:00:00']
Length: 243, dtype: datetime64[ns]

In [41]:
# 문자열 리스트를 pandas DataFrame으로 변환
df = pd.DataFrame(stock_trade_data, columns=['date'])

# 문자열을 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'])

# 각 달의 마지막 날짜 추출
last_days = df.groupby(df['date'].dt.to_period('M')).max()

# DataFrame 형식으로 날짜를 추출
last_days_list = last_days['date'].tolist()


In [15]:
import time
#dt is the last trading day of each month
dt=last_days_list

df_['dt']=pd.to_datetime(df_['dt'])

for i in range(len(dt)): # datetime list가 생성되었는데, 이는 각 달의 마지막 날짜를 의미한다.
    df2 = df_.copy()
    end_data = dt[i]
    start_data = stock_trade_data[stock_trade_data.index(pd.to_datetime(end_data))-(prev_date_num - 1)]
    df2 = df2.loc[df2['dt'] <= end_data]
    df2 = df2.loc[df2['dt'] >= start_data]
    code = sorted(list(set(df2['code'].values.tolist())))
    test_tmp = {}
    for j in tqdm(range(len(code))):
        df3 = df2.loc[df2['code'] == code[j]]
        y = df3[feature_cols].values
        if y.T.shape[1] == prev_date_num:
            test_tmp[code[j]] = y.T
    t1 = time.time()
    result = stock_cor_matrix(test_tmp, list(test_tmp.keys()), prev_date_num, processes=1) 
    # test_tmp: {code: feature} -> code에 대한 feature를 가지고 있는 dict
    # test_tmp.keys(): code list
    # prev_date_num: 20
    # processes: 1 -> 멀티프로세싱을 사용하지 않는다. 차후 멀티프로세싱 사용 시 변경
    result=result.fillna(0)
    print(result.shape)
    print('0000000000')
    print(stock_num)
    for i in range(0,result.shape[0]):
        result.iloc[i,i]=1 # 대각선은 1로 채워준다. -> Adjacency matrix이기 때문에 대각선은 1로 채워주는 작업 assert
    t2 = time.time()
    print('time cost', t2 - t1, 's')
    result.to_csv("./data/relation/"+str(end_data)[:10]+".csv")

100%|██████████| 257/257 [00:10<00:00, 23.38it/s]


(257, 257)
0000000000
368
time cost 10.999049663543701 s


100%|██████████| 273/273 [00:12<00:00, 22.12it/s]


(273, 273)
0000000000
368
time cost 12.351292371749878 s


100%|██████████| 278/278 [00:12<00:00, 21.59it/s]


...

(264, 264)
0000000000
368
time cost 11.535927057266235 s


100%|██████████| 293/293 [00:14<00:00, 20.60it/s]


(293, 293)
0000000000
368
time cost 14.230388641357422 s


100%|██████████| 267/267 [00:11<00:00, 22.56it/s]

(267, 267)
0000000000
368
time cost 11.845136165618896 s


In [16]:
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn as nn
import torch
import math

class GraphAttnMultiHead(Module):
    def __init__(self, in_features, out_features, negative_slope=0.2, num_heads=4, bias=True, residual=True):
        super(GraphAttnMultiHead, self).__init__()
        self.num_heads = num_heads
        self.out_features = out_features
        #MULTIHEAD ATTENTION을 위한 것
        self.weight = Parameter(torch.FloatTensor(in_features, num_heads * out_features)) # in_features -> num_heads*out_features
        self.weight_u = Parameter(torch.FloatTensor(num_heads, out_features, 1))
        self.weight_v = Parameter(torch.FloatTensor(num_heads, out_features, 1))
        self.leaky_relu = nn.LeakyReLU(negative_slope=negative_slope)
        self.residual = residual
        if self.residual:
            self.project = nn.Linear(in_features, num_heads*out_features)
        else:
            self.project = None
        if bias:
            self.bias = Parameter(torch.FloatTensor(1, num_heads * out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(-1))
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)
        self.weight.data.uniform_(-stdv, stdv)
        #Normalizing Term in Attention Weight Calculation
        stdv = 1. / math.sqrt(self.weight_u.size(-1))
        self.weight_u.data.uniform_(-stdv, stdv)
        self.weight_v.data.uniform_(-stdv, stdv)

    def forward(self, inputs, adj_mat, requires_weight=False):
        support = torch.mm(inputs, self.weight)
        support = support.reshape(-1, self.num_heads, self.out_features).permute(dims=(1, 0, 2))
        f_1 = torch.matmul(support, self.weight_u).reshape(self.num_heads, 1, -1)
        f_2 = torch.matmul(support, self.weight_v).reshape(self.num_heads, -1, 1)
        logits = f_1 + f_2
        weight = self.leaky_relu(logits)
        masked_weight = torch.mul(weight, adj_mat).to_sparse()
        attn_weights = torch.sparse.softmax(masked_weight, dim=2).to_dense()
        support = torch.matmul(attn_weights, support)
        support = support.permute(dims=(1, 0, 2)).reshape(-1, self.num_heads * self.out_features)
        if self.bias is not None:
            support = support + self.bias
        if self.residual:
            support = support + self.project(inputs)
        if requires_weight:
            return support, attn_weights
        else:
            return support, None


class PairNorm(nn.Module):    
    '''    
    Ways to prevent oversmoothing in GNNs.
    Source: https://github.com/LingxiaoShawn/PairNorm/blob/master/layers.py

    '''
    def __init__(self, mode='PN', scale=1): # Pair-Normalizaiton, initializing scale to 1
        assert mode in ['None', 'PN', 'PN-SI', 'PN-SCS']
        super(PairNorm, self).__init__()
        self.mode = mode
        self.scale = scale

    def forward(self, x):
        if self.mode == 'None':
            return x
        col_mean = x.mean(dim=0) 
        if self.mode == 'PN':
            x = x - col_mean
            rownorm_mean = (1e-6 + x.pow(2).sum(dim=1).mean()).sqrt()
            x = self.scale * x / rownorm_mean
        if self.mode == 'PN-SI':
            x = x - col_mean
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual
        if self.mode == 'PN-SCS':
            rownorm_individual = (1e-6 + x.pow(2).sum(dim=1, keepdim=True)).sqrt()
            x = self.scale * x / rownorm_individual - col_mean
        return x


class GraphAttnSemIndividual(Module):
    '''
    Node 별로 각 관계에 대한 attention을 구하는 방식. Positive와 negative, 그리고 self로부터 온 hidden state에 대한 attention을 구하게 된다. 
    '''
    
    def __init__(self, in_features, hidden_size=128, act=nn.Tanh()):
        super(GraphAttnSemIndividual, self).__init__()
        self.project = nn.Sequential(nn.Linear(in_features, hidden_size),
                                     act,
                                     nn.Linear(hidden_size, 1, bias=False)) # 같은 차원으로서 보내주기 위함임/
    '''
    Usage:
        self.sem_gat = GraphAttnSemIndividual(in_features=hidden_dim,..)
        support = support.squeeze()
        pos_support, pos_attn_weights = self.pos_gat(support, pos_adj, requires_weight)
        neg_support, neg_attn_weights = self.neg_gat(support, neg_adj, requires_weight)
        support = self.mlp_self(support)
        pos_support = self.mlp_pos(pos_support)
        neg_support = self.mlp_neg(neg_support)
        all_embedding = torch.stack((support, pos_support, neg_support), dim=1)
        all_embedding, sem_attn_weights = self.sem_gat(all_embedding, requires_weight)
    
    '''
    ## Beta는 HeteroGeneous Graph Attention Network의 산출물임.
    # MLP 기반 ATTENTION
    def forward(self, inputs, requires_weight=False):
        w = self.project(inputs) 
        beta = torch.softmax(w, dim=1) # Beta를 구하기
        if requires_weight:
            return (beta * inputs).sum(1), beta
        else:
            return (beta * inputs).sum(1), None


class StockHeteGAT(nn.Module):
    def __init__(self, in_features=5, out_features=8, num_heads=8, hidden_dim=64, num_layers=1):
        super(StockHeteGAT, self).__init__()
        self.encoding = nn.GRU(
            input_size=in_features,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=False,
            dropout=0.1
        )
        self.pos_gat = GraphAttnMultiHead(
            in_features=hidden_dim,
            out_features=out_features,
            num_heads=num_heads
        )
        self.neg_gat = GraphAttnMultiHead(
            in_features=hidden_dim,
            out_features=out_features,
            num_heads=num_heads
        )
        self.mlp_self = nn.Linear(hidden_dim, hidden_dim)
        self.mlp_pos = nn.Linear(out_features*num_heads, hidden_dim)
        self.mlp_neg = nn.Linear(out_features*num_heads, hidden_dim)
        
        self.pn = PairNorm(mode='PN-SI')
        self.sem_gat = GraphAttnSemIndividual(in_features=hidden_dim,
                                              hidden_size=hidden_dim,
                                              act=nn.Tanh())
        self.predictor = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )

        for m in self.modules():
            if isinstance(m, nn.Linear): # Linear layer에 대해서는 xavier_uniform으로 초기화
                nn.init.xavier_uniform_(m.weight, gain=0.02)

    def forward(self, inputs, pos_adj, neg_adj, requires_weight=False):
        _, support = self.encoding(inputs)
        
        '''
        GRU 의 output 형태는 다음과 같다.
        
        
        output: 크기가 (batch_size, sequence_length, hidden_size)인 텐서. 이 출력은 각 시간 스텝의 은닉 상태를 포함하고 있습니다. 이 출력은 사용되지 않습니다.
        support: 크기가 (batch_size, hidden_size)인 텐서. 이것은 GRU의 마지막 은닉 상태를 나타냅니다. 이 출력은 입력 시퀀스에 대한 요약 정보로 간주될 수 있으며, 이후 다른 레이어에 입력으로 사용됩니다.
        
        '''
        support = support.squeeze()
        pos_support, pos_attn_weights = self.pos_gat(support, pos_adj, requires_weight)
        neg_support, neg_attn_weights = self.neg_gat(support, neg_adj, requires_weight)
        support = self.mlp_self(support)
        pos_support = self.mlp_pos(pos_support)
        neg_support = self.mlp_neg(neg_support)
        all_embedding = torch.stack((support, pos_support, neg_support), dim=1)
        all_embedding, sem_attn_weights = self.sem_gat(all_embedding, requires_weight)
        all_embedding = self.pn(all_embedding)
        if requires_weight:
            return self.predictor(all_embedding), (pos_attn_weights, neg_attn_weights, sem_attn_weights)
        else:
            return self.predictor(all_embedding)


In [17]:


import os
import torch
import pickle
import numpy as np
from tqdm import tqdm
import networkx as nx
import pandas as pd
from torch.autograd import Variable


path1 = "./data/valid_csi300.pkl"

df1 = pickle.load(open(path1, 'rb'), encoding='utf-8')
df1.reset_index(inplace=True)
df1.rename(columns={'datetime': 'dt'}, inplace=True)
df1.rename(columns = {'instrument':'code'}, inplace=True)

df1 = df1[['dt', 'code', 'KMID','KLEN',	'KMID2','KUP','KUP2','LABEL0']]


relation = os.listdir('./data/relation/')
relation = sorted(relation) #stock correalation matrix를 담은 걸 가져옴
date_unique=df1['dt'].unique()
#stock_trade_data는 주식이 거래된 내역을 의미함. 
stock_trade_data=date_unique.tolist() #리스트로 만들고 날짜 리스트를 정렬
stock_trade_data.sort()

# 문자열 리스트를 pandas DataFrame으로 변환
df__ = pd.DataFrame(stock_trade_data, columns=['date'])

# 문자열을 datetime 형식으로 변환
df__['date'] = pd.to_datetime(df['date'])

# 각 달의 마지막 날짜 추출
last_days = df__.groupby(df__['date'].dt.to_period('M')).max()
first_days = df__.groupby(df__['date'].dt.to_period('M')).min()
# DataFrame 형식으로 날짜를 추출
last_days_list = last_days['date'].tolist()
first_days_list = first_days['date'].tolist()


df1['dt']=pd.to_datetime(df1['dt']) # datetime으로 바꾸고

def fun(relation_dt, start_dt_month, end_dt_month, df1): # relation_dt가 1월 31일이면, start_dt_month는 1월 1일, end_dt_month는 1월 31일. 앞에 12월이 있으므로!
    prev_date_num = 20 # 즉, 앞에 가져올 수 있는 20일이 있어야 한다는 뜻이다. 

    print('neg_adj over')
    print(neg_adj.shape)
    
    dts = stock_trade_data[stock_trade_data.index(pd.to_datetime(start_dt_month)):stock_trade_data.index(pd.to_datetime(end_dt_month)) + 1]
    print(dts)
    
    for day in dts: # 각 날짜에 대해서 진행
        # 각 시점별로 데이터를 수집한다. 
        adj_all = pd.read_csv('./data/relation/' + relation_dt + '.csv', index_col=0) # relation_dt 자체가 relation 도출하고자 할 때의 날짜
        adj_stock_set = list(adj_all.index)

        '''
        Create Adjacency matrices for Positive and Negative relationships.
        '''

        pos_g = nx.Graph(adj_all > 0.1)  # 0.1보다 큰 것만 connect.
        pos_adj = nx.adjacency_matrix(pos_g).toarray()  # Adjacency matrix로 바꿔줌
        pos_adj = pos_adj - np.diag(np.diag(pos_adj))  # 대각선은 0으로 만들어줌
        pos_adj = torch.from_numpy(pos_adj).type(torch.float32)  # tensor로 바꿔줌

        neg_g = nx.Graph(adj_all < -0.1)  # 상관관계가 -0.1 미만인 것들에 대해서도 testing.
        neg_adj = nx.adjacency_matrix(neg_g)  # Adjancency matrix로 바꿔주기
        neg_adj.data = np.ones(neg_adj.data.shape)  # 1로 바꿔주기 -> threshold 통과 후 0과 1로 바꿔짐
        neg_adj = neg_adj.toarray()  # array로 바꿔주기
        neg_adj = neg_adj - np.diag(np.diag(neg_adj))  # 대각선은 0으로 만들어주기
        neg_adj = torch.from_numpy(neg_adj).type(torch.float32)  # tensor로 바꿔주기
        
        start_data = pd.to_datetime(start_dt_month)
        end_data = pd.to_datetime(end_dt_month)

        df2 = df1.loc[df1['dt'] <= end_data]
        df2 = df2.loc[df2['dt'] >= start_data]
        code = adj_stock_set
        feature_all = []
        mask = []
        labels = []
        day_last_code = []

        for j in range(len(code)):
            df3 = df2.loc[df2['code'] == code[j]]  # code[j]에 해당하는 데이터만 가져옴.
            y = df3[['KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2']].values  # value 가져와서, 여기서는 node feature들을 의미한다.
            
            if y.shape[0] == prev_date_num:  # 20일치 데이터가 다 존재한다면
                feature_all.append(y)  # feature_all에 추가
                mask.append(True)  # mask에 True 추가
                label = df3.loc[df3['dt'] == end_data]['LABEL0'].values  # label은 end_data에 해당하는 label을 가져옴
                labels.append(label[0])  # labels에 추가
                day_last_code.append([code[j], end_data])  # day_last_code에 추가
            else:
                padding_length = prev_date_num - y.shape[0]  # 20일치 데이터가 다 존재하지 않는다면, padding
                if padding_length > 0:
                    y_padded = np.pad(y, ((padding_length, 0), (0, 0)), 'constant', constant_values=0)  # padding
                    feature_all.append(y_padded)  # padding된 데이터를 feature_all에 추가
                else:
                    feature_all.append(y)
                mask.append(False)
                label = df3.loc[df3['dt'] == end_data]['LABEL0'].values
                if label.size > 0:
                    labels.append(label[0])
                else:
                    labels.append(0)
                day_last_code.append([code[j], end_data])

        for i in range(0, len(feature_all)):
            feature_all[i] = feature_all[i][-20:]
        #assert if all elements in feature_all have the same shape
        assert len(set([f.shape for f in feature_all])) == 1
        
        
        feature_all = np.array(feature_all)
        features = torch.from_numpy(feature_all).type(torch.float32)
        assert len(mask) == len(labels)

        labels = torch.tensor(labels, dtype=torch.float32)
        result = {'pos_adj': pos_adj, 'neg_adj': neg_adj, 'features': features, 'labels': labels, 'mask': mask}
        with open('./data/data_train_predict/' + str(end_data)[:10] + '.pkl', 'wb') as f:
            pickle.dump(result, f)  # pickle 파일을 작성
    df = pd.DataFrame(columns=['code', 'dt'], data=day_last_code)  # csv 파일로는 correlation을 저장
    df.to_csv('./data/daily_stock/' + str(end_data)[:10] + '.csv', header=True, index=False, encoding='utf_8_sig')




        
#The first parameter and third parameters indicate the last trading day of each month, and the second parameter indicates the first trading day of each month.



In [27]:
for i in range(len(last_days_list)):
    fun(str(last_days_list[i])[:10], str(first_days_list[i])[:10], str(last_days_list[i])[:10], df1)

neg_adj over
torch.Size([257, 257])
[Timestamp('2017-12-04 00:00:00'), Timestamp('2017-12-05 00:00:00'), Timestamp('2017-12-06 00:00:00'), Timestamp('2017-12-07 00:00:00'), Timestamp('2017-12-08 00:00:00'), Timestamp('2017-12-11 00:00:00'), Timestamp('2017-12-12 00:00:00'), Timestamp('2017-12-13 00:00:00'), Timestamp('2017-12-14 00:00:00'), Timestamp('2017-12-15 00:00:00'), Timestamp('2017-12-18 00:00:00'), Timestamp('2017-12-19 00:00:00'), Timestamp('2017-12-20 00:00:00'), Timestamp('2017-12-21 00:00:00'), Timestamp('2017-12-22 00:00:00'), Timestamp('2017-12-25 00:00:00'), Timestamp('2017-12-26 00:00:00'), Timestamp('2017-12-27 00:00:00'), Timestamp('2017-12-28 00:00:00'), Timestamp('2017-12-29 00:00:00')]
neg_adj over
torch.Size([273, 273])
[Timestamp('2018-01-02 00:00:00'), Timestamp('2018-01-03 00:00:00'), Timestamp('2018-01-04 00:00:00'), Timestamp('2018-01-05 00:00:00'), Timestamp('2018-01-08 00:00:00'), Timestamp('2018-01-09 00:00:00'), Timestamp('2018-01-10 00:00:00'), Timestam

In [19]:
import os
import sys
from torch.utils import data
import pickle

class AllGraphDataSampler(data.Dataset):
    '''
    base_dir = "./data/data_train_predict/"
    
    '''
    
    def __init__(self, base_dir, gname_list=None, data_start=None, data_middle=None, data_end=None, mode="train"):
        self.data_dir = os.path.join(base_dir)
        self.mode = mode
        self.data_start = data_start
        self.data_middle = data_middle
        self.data_end = data_end
        if gname_list is None:
            self.gnames_all = os.listdir(self.data_dir)
            self.gnames_all.sort()
        if mode == "train":
            self.gnames_all = self.gnames_all[self.data_start:self.data_middle] # 데이터 파일의 이름을 저장하는 리스트 -> train이므로 start~middle 까지 가져온다
        elif mode == "val":
            self.gnames_all = self.gnames_all[self.data_middle:self.data_end] # 데이터 파일의 이름을 저장하는 리스트 -> val이므로 middle~end
        self.data_all = self.load_state()

    def __len__(self):
        return len(self.data_all)

    def load_state(self):
        data_all = []
        length = len(self.gnames_all)
        for i in range(length):
            sys.stdout.flush()
            sys.stdout.write('{} data loading: {:.2f}%{}'.format(self.mode, i*100/length, '\r'))
            data_all.append(pickle.load(open(os.path.join(self.data_dir, self.gnames_all[i]), "rb")))
        print('{} data loaded!'.format(self.mode))
        return data_all

    def __getitem__(self, idx):
        return self.data_all[idx]

In [21]:
from trainer import *
from model import *

from trainer.trainer import *
#from data_loader import *
from model.Thgnn import *
import warnings
import torch
import os
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import pandas as pd
from pandas.core.frame import DataFrame
from tqdm import tqdm

warnings.filterwarnings("ignore")
t_float = torch.float64
torch.multiprocessing.set_sharing_strategy('file_system')

class Args:
    def __init__(self, gpu=0, subtask="regression"):
        # device
        self.gpu = str(gpu)
        self.device = 'cpu'
        # data settings
        adj_threshold = 0.1 # noise를 줄이기 위한 상관관계 threshold
        self.adj_str = str(int(100*adj_threshold))
        self.pos_adj_dir = "pos_adj_" + self.adj_str # 미사용
        self.neg_adj_dir = "neg_adj_" + self.adj_str # 미사용
        self.feat_dir = "features"
        self.label_dir = "label"
        self.mask_dir = "mask"
        self.data_start = data_start
        self.data_middle = data_middle
        self.data_end = data_end
        self.pre_data = pre_data
        # epoch settings
        self.max_epochs = 60 #핛습 횟수
        self.epochs_eval = 10 #몇 번마다 평가할지
        # learning rate settings
        self.lr = 0.0002 # learning rate
        self.gamma = 0.3 #  learning rate decay
        # model settings
        self.hidden_dim = 128 #  hidden dimension
        self.num_heads = 8 
        self.out_features = 32
        self.model_name = "StockHeteGAT"
        self.batch_size = 1
        self.loss_fcn = mse_loss
        # save model settings
        self.save_path = os.path.join(os.path.abspath('.'), "./data/model_saved/")
        self.load_path = self.save_path
        self.save_name = self.model_name + "_hidden_" + str(self.hidden_dim) + "_head_" + str(self.num_heads) + \
                         "_outfeat_" + str(self.out_features) + "_batchsize_" + str(self.batch_size) + "_adjth_" + \
                         str(self.adj_str)
        self.epochs_save_by = 10
        self.sub_task = subtask
        eval("self.{}".format(self.sub_task))()

    def regression(self):
        self.save_name = self.save_name + "_reg_rank_"
        self.loss_fcn = mse_loss
        self.label_dir = self.label_dir + "_regression"
        self.mask_dir = self.mask_dir + "_regression"

    def regression_binary(self):
        self.save_name = self.save_name + "_reg_binary_"
        self.loss_fcn = mse_loss
        self.label_dir = self.label_dir + "_twoclass"
        self.mask_dir = self.mask_dir + "_twoclass"

    def classification_binary(self):
        self.save_name = self.save_name + "_clas_binary_"
        self.loss_fcn = bce_loss
        self.label_dir = self.label_dir + "_twoclass"
        self.mask_dir = self.mask_dir + "_twoclass"

    def classification_tertiary(self):
        self.save_name = self.save_name + "_clas_tertiary_"
        self.loss_fcn = bce_loss
        self.label_dir = self.label_dir + "_threeclass"
        self.mask_dir = self.mask_dir + "_threeclass"


def fun_train_predict(data_start, data_middle, data_end, pre_data):
    args = Args()
    os.environ["CUDA_VISIBLE_DEVICES"] = str(args.gpu)
    
    # 그래프 만들고!!!!#### 
    
    
    dataset = AllGraphDataSampler(base_dir="./data/data_train_predict/", data_start=data_start,
                                  data_middle=data_middle, data_end=data_end)
    val_dataset = AllGraphDataSampler(base_dir="./data/data_train_predict/", mode="val", data_start=data_start,
                                      data_middle=data_middle, data_end=data_end)
    dataset_loader = DataLoader(dataset, batch_size=args.batch_size, pin_memory=True, collate_fn=lambda x: x)
    val_dataset_loader = DataLoader(val_dataset, batch_size=1, pin_memory=True)
    
    
    model = eval(args.model_name)(hidden_dim=args.hidden_dim, num_heads=args.num_heads,
                                  out_features=args.out_features).to(args.device)

    # training 과정
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    cold_scheduler = StepLR(optimizer=optimizer, step_size=5000, gamma=0.9, last_epoch=-1)
    default_scheduler = cold_scheduler
    print('start training')
    for epoch in range(args.max_epochs):
        train_loss = train_epoch(epoch=epoch, args=args, model=model, dataset_train=dataset_loader,
                                 optimizer=optimizer, scheduler=default_scheduler, loss_fcn=mse_loss)
        if epoch % args.epochs_eval == 0:
            eval_loss, _ = eval_epoch(args=args, model=model, dataset_eval=val_dataset_loader, loss_fcn=mse_loss)
            print('Epoch: {}/{}, train loss: {:.6f}, val loss: {:.6f}'.format(epoch + 1, args.max_epochs, train_loss,
                                                                              eval_loss))
        else:
            print('Epoch: {}/{}, train loss: {:.6f}'.format(epoch + 1, args.max_epochs, train_loss))
        if (epoch + 1) % args.epochs_save_by == 0:
            print("save model!")
            state = {'model': model.state_dict(), 'optimizer': optimizer.state_dict(), 'epoch': epoch + 1}
            torch.save(state, os.path.join(args.save_path, pre_data + "_epoch_" + str(epoch + 1) + ".dat"))

    # predicting 과정
    checkpoint = torch.load(os.path.join(args.load_path, pre_data + "_epoch_" + str(epoch + 1) + ".dat"))
    model.load_state_dict(checkpoint['model'])
    data_code = os.listdir('./data/daily_stock')
    data_code = sorted(data_code)
    data_code_last = data_code[data_middle:data_end]
    df_score=pd.DataFrame()
    for i in tqdm(range(len(val_dataset))):
        df = pd.read_csv('./data/daily_stock/' + data_code_last[i], dtype=object)
        tmp_data = val_dataset[i]
        pos_adj, neg_adj, features, labels = extract_data(tmp_data, args.device)
        model.train()
        logits = model(features, pos_adj, neg_adj)
        result = logits.data.cpu().numpy().tolist()
        result_new = []
        for j in range(len(result)):
            result_new.append(result[j][0])
        res = {"score": result_new}
        res = DataFrame(res)
        df['score'] = res
        df_score=pd.concat([df_score,df])

        #df.to_csv('prediction/' + data_code_last[i], encoding='utf-8-sig', index=False)
    df_score.to_csv('./data/prediction/pred.csv')
    print(df_score)
    
if __name__ == "__main__":
    data_start = 20
    data_middle = 40
    data_end = data_middle+4
    pre_data = '2018_FIRST'
    fun_train_predict(data_start, data_middle, data_end, pre_data)

train data loaded!: 95.00%
val data loaded!: 75.00%


KeyboardInterrupt: 

In [22]:
from torch.utils.data import DataLoader

data_start = 20
data_middle = 60
data_end = data_middle+4

dataset = AllGraphDataSampler(base_dir="./data/data_train_predict/", data_start=data_start,
                                  data_middle=data_middle, data_end=data_end)
val_dataset = AllGraphDataSampler(base_dir="./data/data_train_predict/", mode="val", data_start=data_start,
                                      data_middle=data_middle, data_end=data_end)
dataset_loader = DataLoader(dataset, batch_size=1, pin_memory=True, collate_fn=lambda x: x)
val_dataset_loader = DataLoader(val_dataset, batch_size=1, pin_memory=True)

train data loaded!: 97.50%
val data loaded!: 75.00%


In [23]:
for data in dataset:
    break

{'pos_adj': tensor([[0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 1.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         ...,
         [1., 1., 0.,  ..., 0., 1., 1.],
         [0., 0., 1.,  ..., 1., 0., 1.],
         [0., 1., 0.,  ..., 1., 1., 0.]]),
 'neg_adj': tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 'features': tensor([[[ 1.1622e+00,  2.0880e-02,  1.0539e+00,  2.2625e-01,  1.2058e-01],
          [-7.3244e-01, -5.5122e-01, -9.7127e-01, -5.1207e-01, -3.9061e-01],
          [ 4.6380e-02, -1.2872e+00,  1.5177e-01, -1.4119e-01,  1.9531e+00],
          ...,
          [-2.8601e-01, -1.1187e+00, -7.0045e-01, -2.4247e-01,  9.1806e-01],
          [ 3.3580e-01, -1.2181e+00,  9.6573e-01, -6.1477e-01,  9.8673e-02],
          [ 5.2392e-01, -9.1320e-01,  9.81

In [26]:
data['pos_adj'].shape

torch.Size([273, 273])

In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = eval('StockHeteGAT')(hidden_dim=128, num_heads=8, out_features = 32).to(device)

c:\Users\admin\miniconda3\envs\dongwoo\lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
optimizer = optim.Adam(model.parameters(), lr=args.lr)
cold_scheduler = StepLR(optimizer=optimizer, step_size=5000, gamma=0.9, last_epoch=-1)



torch.Size([4, 20, 6])